In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 500)

In [20]:
ALL_EVALUATIONS="tests/evaluation_05_retrievers_000_009_sampled_50_queries_gpt-4-1106-preview_20231215.tsv"

BM25_RUN="tests/200-queries-passagens-by-BM25-pt_v2_with_query_id_run.txt"
BM25_E5_RUN="tests/bm25_e5_docs_000_999_50_sampled_queries_run.txt"
BM25_MT5_RUN="tests/200-queries-passagens-by-BM25-mT5_v4_with_query_id_run.txt"
SPLADE_RUN="tests/splade_pt_result_with_clueweb_id_run.txt"
COLBERTX_RUN="tests/colbertx_all_znormalized_validation_scores_000_199_run.txt"

In [21]:
retrievers_results = []

### Format GPT-4 evaluations to qrels format

In [22]:
eval_df = pd.read_csv(ALL_EVALUATIONS, sep='\t')

In [23]:
eval_df

,query_id,query,passage_id,mT5,splade,colbertx,bm25_e5,bm25,passage,score,reasoning,usage,cost,duration,saved_cost
0,1,Qual a maior característica da fauna brasileira?,clueweb22-pt0000-82-13187_0,False,False,True,False,False,"Mamíferos do Brasil - Zoologia | Museu Nacional - UFRJ ➤ Início ➤ Exposições ➤ Zoologia (Acervo) ➤ Insetos ➤ Mamíferos do Brasil Mamíferos do Brasil Uma larga coleção de mamíferos da fauna brasileira. Entre os importantes representantes deste acervo de espécimes taxidermizados encontram-se grandes símbolos da diversidade biológica do Brasil. Bugio, Barbado, Guariba Bugio Cachorro-do-mato Caititu Capivara Caxinguelê, Serelepe Cervo-do-Pantanal Gato-do-Mato-pequeno Irara Lobo-Guará Lontra Mão-Pelada Muriqui Onça-Pintada Ouriço-Cacheiro, Cuandu Paca Preguiça-de-Bentinho Preguiça-de-três-dedos Rato-Coró Saruê, Gambá Sauá, Guigó Tamanduá-bandeira Tamanduá-í Tamanduá-mirim, meleta Tatu-Canastra Tatu-de-rabo-mole Tatu-Galinha Tatu-Peludo Veado Veado-mateiro « Voltar | Voltar ao Topo",1,"A passagem lista vários mamíferos da fauna brasileira, mas não descreve características gerais da fauna brasileira, como a pergunta solicita. A passagem está relacionada ao tema da fauna brasileira, mas não responde diretamente à pergunta sobre a maior característica da fauna.","{'prompt_tokens': 765, 'completion_tokens': 75, 'total_tokens': 840}",0.00000,6.693229,0.02745
1,1,Qual a maior característica da fauna brasileira?,clueweb22-pt0000-59-04599_2,False,False,True,False,False,"Segundo uma lista levantada pelo Comitê Brasileiro de Registros Ornitológicos (CBRO), algumas das principais espécies encontradas aqui no país são: Bem-te-vi, Arara-azul, Tuiuiú, Tucano e o Papagaio. Só no Cerrado pode-se encontrar por volta de 148 espécies de aves bem específicas da região. Algumas das aves que mais representam esta região brasileira são: Seriemas; Pavãozinho-do-pará; Coruja-orelhuda; Arara-canindé; Coruja-da-igreja; Arara-vermelha-grande; Tuiuiú (considerado o símbolo do Pantanal). Quando chegamos à região nordestina a ave brasileira já foi até tema de música, a famosa ave com seu voo incrível chamada “Asa branca”, ave que deu nome à música de mais sucesso na voz do cantor e compositor Luiz Gonzaga. A Amazônia é uma região brasileira que qualquer um se encanta pelas belezas naturais principalmente quando o assunto são espécies de aves. As araras com sua pelagem colorida chamam a atenção de todos e podem ser vistas por todos os lados na região. A segunda ave que pode ser encontrada facilmente na Amazônia é o tucano, com suas características marcantes, um bico longo e cores vibrantes que marcam a região dos olhos. Claro que podemos encontrar essas bela espécie de ave em zoológicos espalhados por todo o país ou em áreas de matas em geral.",1,"A passagem descreve várias espécies de aves encontradas no Brasil e suas características, mas não resume qual é a maior característica da fauna brasileira de forma geral. A pergunta pede uma característica ampla da fauna, enquanto a passagem foca em exemplos específicos de aves em diferentes regiões.","{'prompt_tokens': 860, 'completion_tokens': 82, 'total_tokens': 942}",0.00000,5.163547,0.03072
2,1,Qual a maior característica da fauna brasileira?,clueweb22-pt0000-64-06285_39,True,True,False,False,False,"A exuberante fauna e flora brasileiras, relatadas por Pero Vaz de Caminha em sua carta à Coroa Portuguesa, está sendo destruída gradativamente pela ação antrópica, por meio do desmatamento e da caça d... Leia redação completa Redação sem título. 02/07/2016 O Brasil tem a maior biodiversidade do mundo. Devido a sua variedade de espécies silvestres, desencadeou uma enorme ambição global ao tráfico de animais desde o período colonial. Atualmente, tal ato a... Leia redação completa Redação sem título. 19/09/2018 O Brasil com seu vasto território é um dos países com a maior biodiversidade de animais silvestres. Entretanto, não é dado a devida importância a essa característica, já que é historicamente mar

In [24]:
eval_df.drop_duplicates("query_id")[['query_id', 'query']]

,query_id,query
0,1,Qual a maior característica da fauna brasileira?
35,2,Por que os países Guiana e Suriname não são filiados a Conmebol?
71,9,quais países europeus seguem o regime monarquista?
108,11,quais os critérios de definição dos monumentos intitulados maravilhas do mundo moderno?
135,13,Como o Brasil reagiu a epidemia de AIDS no fim do século XX?
166,15,Qual a maior torcida de futebol do Brasil?
189,17,Quando se realizou o plebiscito popular para definir o sistema político do Brasil?
217,20,Por que a legislação de um país é tão importante?
260,21,Qual o nosso atual sistema de governo?
301,22,Qual o sistema de governo da África do Sul?


#### Check scores distribution on the current qrels

In [25]:
np.unique(eval_df['score'].to_numpy(), return_counts=True)

(array([0, 1, 2, 3]), array([600, 327, 293, 388]))

In [26]:
for group_name, group_df in eval_df.groupby('query_id'):
    print("\nquery_id={}".format(group_name))
    
    print(np.unique(group_df['score'].to_numpy(), return_counts=True))


query_id=1
(array([0, 1, 2, 3]), array([ 5, 17,  8,  5]))

query_id=2
(array([0, 1, 2, 3]), array([31,  2,  1,  2]))

query_id=9
(array([0, 1, 2, 3]), array([24,  3,  8,  2]))

query_id=11
(array([0, 1, 2, 3]), array([13,  6,  6,  2]))

query_id=13
(array([0, 1, 2, 3]), array([12,  8,  7,  4]))

query_id=15
(array([0, 1, 2, 3]), array([ 1, 10,  1, 11]))

query_id=17
(array([0, 1, 2, 3]), array([ 4,  1,  1, 22]))

query_id=20
(array([0, 1, 2, 3]), array([ 4, 23, 13,  3]))

query_id=21
(array([0, 1, 2, 3]), array([18,  9,  1, 13]))

query_id=22
(array([0, 1, 3]), array([19,  6,  4]))

query_id=24
(array([0, 1, 2, 3]), array([14, 15,  8,  2]))

query_id=26
(array([0, 1, 2, 3]), array([ 1,  2,  6, 19]))

query_id=28
(array([0, 1, 2, 3]), array([16,  6,  8,  4]))

query_id=47
(array([0, 1, 2]), array([14, 13, 12]))

query_id=49
(array([0, 1, 2, 3]), array([11,  9, 13,  1]))

query_id=51
(array([0, 1, 2, 3]), array([ 8, 10,  4,  1]))

query_id=54
(array([1, 2, 3]), array([ 5,  7, 24]))

que

In [27]:
eval_df.groupby('query_id').count()

,query,passage_id,mT5,splade,colbertx,bm25_e5,bm25,passage,score,reasoning,usage,cost,duration,saved_cost
query_id,,,,,,,,,,,,,,
1,35,35,35,35,35,35,35,35,35,35,35,35,35,35
2,36,36,36,36,36,36,36,36,36,36,36,36,36,36
9,37,37,37,37,37,37,37,37,37,37,37,37,37,37
11,27,27,27,27,27,27,27,27,27,27,27,27,27,27
13,31,31,31,31,31,31,31,31,31,31,31,31,31,31
15,23,23,23,23,23,23,23,23,23,23,23,23,23,23
17,28,28,28,28,28,28,28,28,28,28,28,28,28,28
20,43,43,43,43,43,43,43,43,43,43,43,43,43,43
21,41,41,41,41,41,41,41,41,41,41,41,41,41,41


#### Prepare the qrels format for trec_eval tools

In [28]:
eval_df['0'] = '0'

In [29]:
qrels_filename = os.path.join(os.path.dirname(ALL_EVALUATIONS), "{}_qrel.tsv".format(os.path.splitext(os.path.basename(ALL_EVALUATIONS))[0]))

In [30]:
qrels_filename

'tests/evaluation_05_retrievers_000_009_sampled_50_queries_gpt-4-1106-preview_20231215_qrel.tsv'

In [31]:
eval_df.sort_values('query_id')[['query_id', '0', 'passage_id', 'score']].to_csv(qrels_filename, header=None, index=False, sep=" ")

### Compute BM25 NDCG@10

In [32]:
result = !/media/eduseiti/bigdata01/unicamp/ia368v_dd/pyserini/tools/eval/trec_eval.9.0.4/trec_eval -c -mndcg_cut.10 \
    {qrels_filename} tests/200-queries-passagens-by-BM25-pt_v2_with_query_id_run.txt

In [33]:
retrievers_results.append({'retriever':'BM25', 'nDCG@10': result[0].split('\t')[-1]})

In [34]:
retrievers_results

[{'retriever': 'BM25', 'nDCG@10': '0.4705'}]

### Compute BM25 + E5 NDCG@10

In [35]:
result = !/media/eduseiti/bigdata01/unicamp/ia368v_dd/pyserini/tools/eval/trec_eval.9.0.4/trec_eval -c -mndcg_cut.10 \
    {qrels_filename} {BM25_E5_RUN}

In [36]:
retrievers_results.append({'retriever':'BM25+E5', 'nDCG@10': result[0].split('\t')[-1]})

### Compute BM25 + mT5 NDCG@10

In [37]:
result = !/media/eduseiti/bigdata01/unicamp/ia368v_dd/pyserini/tools/eval/trec_eval.9.0.4/trec_eval -c -mndcg_cut.10 \
    {qrels_filename} {BM25_MT5_RUN}

In [38]:
retrievers_results.append({'retriever':'BM25+mT5', 'nDCG@10': result[0].split('\t')[-1]})

### Compute SPLADE NDCG@10

In [39]:
result = !/media/eduseiti/bigdata01/unicamp/ia368v_dd/pyserini/tools/eval/trec_eval.9.0.4/trec_eval -c -mndcg_cut.10 \
    {qrels_filename} {SPLADE_RUN}

In [40]:
retrievers_results.append({'retriever':'SPLADE_pt-BR', 'nDCG@10': result[0].split('\t')[-1]})

### Compute ColBERTX NDCG@10

In [41]:
result = !/media/eduseiti/bigdata01/unicamp/ia368v_dd/pyserini/tools/eval/trec_eval.9.0.4/trec_eval -c -mndcg_cut.10 \
    {qrels_filename} {COLBERTX_RUN}

In [42]:
retrievers_results.append({'retriever':'ColBERT-X_mMARCO_pt-BR', 'nDCG@10': result[0].split('\t')[-1]})

### Check the overall results

In [43]:
pd.DataFrame(retrievers_results)

,retriever,nDCG@10
0,BM25,0.4705
1,BM25+E5,0.6738
2,BM25+mT5,0.7525
3,SPLADE_pt-BR,0.6062
4,ColBERT-X_mMARCO_pt-BR,0.6596
